<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.  select all (根據filter(最終展示的) 勾選)  + 圖表

p.s. 一定要用 唯一數字index 做select_rows

In [ ]:
import dash
from dash import Dash, dash_table, html, dcc, Output, Input, State
import pandas as pd
import plotly.express as px

# 假資料 + 原始索引欄
df = pd.DataFrame({
    "re_id": ['re_001','re_002', 're_003', 're_004', 're_005'],
    "姓名": ["Alice", "Bob", "Charlie", "David", 'Cindy'],
    "年齡": ['25', '30', '35', '40', '30'],
    "城市": ["台北", "高雄", "台中", "台南", "台北"]
})
### 每個 row 需要一個'獨立index'  (需用這個!因為select_rows要數字，不能是String(ex:re_id))
df['_index'] = df.index

app = Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id='select-all',
        options=[{'label': 'Select All (filtered)', 'value': 'all'}],
        value=[],
        style={'marginBottom': '10px'}
    ),
    dash_table.DataTable(
        id='my-table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        row_selectable="multi",
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[]
    ),
    html.Br(),

    # ✅ 圖（放在 DataTable 下方）
    html.Div([
        dcc.Graph(id='pareto-chart', style={'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(id='pie-chart', style={'width': '50%', 'display': 'inline-block'})
    ])])

# ✅ 控制勾選：根據篩選後資料決定 selected_rows（回傳原始 index）
# 1. 取得「篩選後」資料（來自 derived_virtual_data)
# 2. 找出這些資料在原始資料中的 index
# 3. 回傳這些 _index，正確設定 selected_row (原始資料的)
@app.callback(
    Output('my-table', 'selected_rows'),
    Input('select-all', 'value'),
    State('my-table', 'derived_virtual_data'),
    State('my-table', 'data')
)
def select_filtered_rows(select_all_value, filtered_data, full_data):
    if 'all' in select_all_value and filtered_data is not None:
        return [row['_index'] for row in filtered_data]
    return []

# ✅ 底色樣式：根據篩選資料中的原始 index 與勾選狀態設定底色
# (透過Input: derived_virtual_data，讓 Dash 只要一變動就觸發底色更新 callback，確保樣式正確。)
@app.callback(
    Output('my-table', 'style_data_conditional'),
    Input('my-table', 'derived_virtual_data'),
    Input('my-table', 'selected_rows')
)
def update_row_style(filtered_data, selected_rows):
    if not selected_rows or not filtered_data:
        return []

    return [
        {
            'if': {'row_index': i},
            'backgroundColor': '#ffe6f0',
            'border': '1px solid #ff99cc'
        }
        for i, row in enumerate(filtered_data)
        if row.get('_index') in selected_rows
    ]


# ✅ Pareto + Pie 圖 callback
@app.callback(
    Output('pareto-chart', 'figure'),
    Output('pie-chart', 'figure'),
    Input('my-table', 'derived_virtual_data')
)
def update_charts(filtered_data):
    dff = pd.DataFrame(filtered_data) if filtered_data else df

    # ▓ Pareto
    city_counts = dff["城市"].value_counts().reset_index()
    city_counts.columns = ["城市", "數量"]
    city_counts["累積百分比"] = city_counts["數量"].cumsum() / city_counts["數量"].sum() * 100

    pareto_fig = go.Figure()
    pareto_fig.add_trace(go.Bar(
        x=city_counts["城市"],
        y=city_counts["數量"],
        name="數量",
        marker_color='lightsalmon'
    ))
    pareto_fig.add_trace(go.Scatter(
        x=city_counts["城市"],
        y=city_counts["累積百分比"],
        name="累積百分比",
        yaxis='y2',
        mode='lines+markers',
        marker=dict(color='blue')
    ))
    pareto_fig.update_layout(
        title="Pareto 圖：城市人數分布",
        yaxis=dict(title="數量"),
        yaxis2=dict(title="累積百分比", overlaying='y', side='right', range=[0, 110]),
        xaxis=dict(title="城市"),
        margin=dict(t=40, b=40)
    )

    # ▓ Pie
    pie_fig = go.Figure(
        go.Pie(
            labels=city_counts["城市"],
            values=city_counts["數量"],
            marker=dict(colors=px.colors.qualitative.Set3),  # 可以替換成 Set1, Pastel1, Dark2 等
            hole=0.3,
        )
    )
    pie_fig.update_layout(title="Pie 圖：城市分布")

    return pareto_fig, pie_fig


if __name__ == '__main__':
    app.run(debug=True)

### 2. 被select的資料， 找出其row後 撈出其Re_id

In [ ]:
# 被select的那些row的 re_id
selected_re_ids = df_all.loc[df_all['_index'].isin(selected_rows), 're_id'].tolist()

In [ ]:
# 新增 callback 觸發下載
@app.callback(
    Output("download-data", "data"),
    Input("btn-download", "n_clicks"),
    State("my-table", "selected_rows"),
    State("stored-data", "data"),
    prevent_initial_call=True,
)
def generate_excel(n_clicks, selected_rows, stored_data):
    if not selected_rows:
        return dash.no_update  # 沒選資料就不下載

    # 把 stored_data 轉 DataFrame（包含隱藏欄位）
    df_all = pd.DataFrame(stored_data)

    # 選出被選中的資料
    df_selected = df_all[df_all['_index'].isin(selected_rows)]

    # 用 send_data_frame 輸出 Excel 檔
    return send_data_frame(df_selected.to_excel, "selected_data.xlsx", index=False)

### 3. table 有的欄位不show & markdown

In [ ]:
columns = [
    {
        "name": i,
        "id": i,
        "presentation": "markdown" if i in ["城市", "姓名"] else "input"
    }
    for i in df.columns if i not in ('re_id', '_index', 'info')
]
